In [ ]:
import scvelo as scv
import scanpy as sc
import cellrank as cr
import numpy as np
import pandas as pd
import anndata as ad
import loompy
import matplotlib as plt
import igraph

In [ ]:
# import CellRank kernels and estimators
from cellrank.kernels import ConnectivityKernel
from cellrank.kernels import CytoTRACEKernel
from cellrank.kernels import VelocityKernel
from cellrank.estimators import GPCCA

In [ ]:
scv.settings.verbosity = 3
scv.settings.set_figure_params('scvelo', transparent=False, facecolor='white', dpi=200, dpi_save=500, frameon=False)
cr.settings.verbosity = 2

In [ ]:
adata = sc.read_h5ad("F:/Gekko_Gecko/SingleCell_Analysis/SC_Analysis_Seurat/allET.integrated_celltypes_filt.h5ad")
adata

In [ ]:
ldata1 = scv.read('F:/Gekko_Gecko/SingleCell_Analysis/velodata/ET_3dpo.loom', cache=True)
ldata2 = scv.read('F:/Gekko_Gecko/SingleCell_Analysis/velodata/ET_7dpo.loom', cache=True)

In [ ]:
barcodes = [bc.split(':')[1] for bc in ldata1.obs.index.tolist()]
barcodes = ['ET_3dpo_' + bc[0:len(bc)-1] + '-1' for bc in barcodes]
ldata1.obs.index = barcodes
ldata1.obs.index

In [ ]:
barcodes = [bc.split(':')[1] for bc in ldata2.obs.index.tolist()]
barcodes = ['ET_7dpo_' + bc[0:len(bc)-1] + '-1' for bc in barcodes]
ldata2.obs.index = barcodes
ldata2.obs.index

In [ ]:
# make variable names unique
ldata1.var_names_make_unique()
ldata2.var_names_make_unique()

In [ ]:
adata.obs.index

In [ ]:
# concatenate the six loom
ldata_ET = ldata1.concatenate([ldata2])

In [ ]:
#clean the sample barcodes and save the sample name to sampel_batch
scv.utils.clean_obs_names(adata)
scv.utils.clean_obs_names(ldata_ET)

In [ ]:
adata_merge = scv.utils.merge(adata, ldata_ET)
adata_merge

In [ ]:
#adata_merge.__dict__['_raw'].__dict__['_var'] = adata_merge.__dict__['_raw'].__dict__['_var'].rename(columns={'_index': 'features'})
#adata_merge.write('F:/Gekko_Gecko/SingleCell_Analysis/SC_Analysis_Seurat/adataveloraw_ET.h5ad')

In [ ]:
#adata_merge = sc.read_h5ad("F:/Gekko_Gecko/SingleCell_Analysis/SC_Analysis_Seurat/adataveloraw_ET.h5ad")
#adata_merge

In [ ]:
sc.pl.umap(adata_merge, color='celltypes', frameon=False, legend_loc='right margin', title='', save=None)

In [ ]:
scv.pl.proportions(adata_merge, groupby='celltypes', layers=None, highlight='unspliced', add_labels_pie=True, add_labels_bar=True, fontsize=8, figsize=(10, 2), dpi=300, use_raw=True, show=True, save="spliced_proportion.png")

In [ ]:
sc.pp.neighbors(adata_merge)

In [ ]:
sc.tl.draw_graph(adata_merge, layout='fr')
sc.tl.draw_graph(adata_merge, layout='drl')
#sc.tl.draw_graph(adata_merge, layout='kk')
sc.tl.draw_graph(adata_merge, layout='lgl')
#sc.tl.draw_graph(adata_merge, layout='rt')
sc.tl.diffmap(adata_merge)

In [ ]:
adata_merge.obsm

In [ ]:
sc.pl.embedding(
    adata_merge,
    basis="draw_graph_lgl",
    color=["orig.ident", "celltypes"],
    color_map="gnuplot", save="ET_force-directed_lgl.png" ,
)

In [ ]:
sc.pl.embedding(
    adata_merge,
    basis="draw_graph_fr",
    color=["orig.ident", "celltypes"],
    color_map="gnuplot", save="ET_force-directed_fr.png",
)

In [ ]:
sc.pl.embedding(
    adata_merge,
    basis="draw_graph_drl",
    color=["orig.ident", "celltypes"],
    color_map="gnuplot", save="ET_force-directed_drl.png",
)

In [ ]:
sc.pl.scatter(
    adata_merge,
    basis="diffmap",
    color=["orig.ident", "celltypes"],
    color_map="gnuplot", save="ET_diffmap.png", 
)

In [ ]:
scv.pp.filter_and_normalize(adata_merge)

In [ ]:
scv.pp.moments(adata_merge)

In [ ]:
##scv.tl..recover_dynamics(adata_merge)

In [ ]:
scv.tl.velocity(adata_merge, mode = "stochastic")

In [ ]:
vk = VelocityKernel(adata_merge)

In [ ]:
vk.compute_transition_matrix()
print(vk)

In [ ]:
vk.plot_projection(basis="draw_graph_drl", color=["celltypes","orig.ident"], title="velocity stream of regenerating tail", 
    save="ET_embedstream_drl_celltype_velokernel1.png")

In [ ]:
vk.plot_projection(basis="draw_graph_fr", color=["celltypes","orig.ident"], title="velocity stream of regenerating tail", 
    save="ET_embedstream_fr_celltype_velokernel1.png")

In [ ]:
vk.plot_projection(basis="draw_graph_lgl", color=["celltypes","orig.ident"], title="velocity stream of regenerating tail", legend_loc="best",
    save="ET_embedstream_lgl_celltype_velokernel1.png")

In [ ]:
vk.plot_projection(basis="diffmap", color=["celltypes","orig.ident"], title="velocity stream of regenerating tail", legend_loc="best",
    save="ET_embedstream_diffmap_celltype_velokernel1.png")

In [ ]:
vk.plot_projection(basis="umap", color=["celltypes","orig.ident"], title="velocity stream of regenerating tail",
    save="ET_embedstream_umap_celltype_velokernel1.png")

In [ ]:
vk.plot_projection(basis="umap", color="celltypes", title="velocity stream of regenerating tail", legend_fontsize="x-small",
    save="ET_embedstream_umap_celltype_velokernel.png")

In [ ]:
vk.plot_projection(basis="draw_graph_lgl", color="celltypes", title="velocity stream of regenerating tail", legend_loc="best",
    save="ET_embedstream_lgl_celltype_velokernel.png")

In [ ]:
vk.plot_projection(basis="diffmap", color="celltypes", title="velocity stream of regenerating tail", legend_loc="best",
    save="ET_embedstream_diffmap_celltype_velokernel.png")

In [ ]:
vk.plot_projection(basis="draw_graph_drl", color="celltypes", title="velocity stream of regenerating tail",
    save="ET_embedstream_drl_celltype_velokernel.png")

In [ ]:
vk.plot_random_walks(
    n_sims=100,
    start_ixs={"celltypes": "Tailbud mesenchymal cells"},
    basis="umap",
    color="celltypes",
    legend_loc="right",
    seed=1, save="ET_randomwalk_celltype_velokernel.png", title="random walk simulation on embryonic tail",
)

In [ ]:
vk.plot_random_walks(
    n_sims=100,
    start_ixs={"celltypes": "Tailbud mesenchymal cells"},
    basis="draw_graph_drl",
    color="celltypes",
    legend_loc="right",
    seed=1, save="ET_randomwalk_drl_celltype_velokernel.png", title="random walk simulation on embryonic tail",
)

In [ ]:
vk.plot_random_walks(
    n_sims=100,
    start_ixs={"celltypes": "Tailbud mesenchymal cells"},
    basis="draw_graph_lgl",
    color="celltypes",
    legend_loc="right",
    seed=1, save="ET_randomwalk_lgl_celltype_velokernel.png", title="random walk simulation on embryonic tail",
)

In [ ]:
vk.plot_random_walks(
    n_sims=100,
    start_ixs={"celltypes": "Tailbud mesenchymal cells"},
    basis="diffmap",
    color="celltypes",
    legend_loc="right",
    seed=1, save="ET_randomwalk_diffmap_celltype_velokernel.png", title="random walk simulation on embryonic tail",
)

In [ ]:
#adata_merge.obsm['X_diffmap'][:, 3].argmax()

In [ ]:
#root_ixs = 7866  # has been found using `adata.obsm['X_diffmap'][:, 3].argmax()`
#scv.pl.scatter(
#    adata_merge,
#    basis="diffmap",
#    c=["celltypes", root_ixs],
#    legend_loc="right",
#    components=["2, 3"],
#)

#adata_merge.uns["iroot"] = root_ixs

In [ ]:
g_fwd = cr.estimators.GPCCA(vk)
print(g_fwd)

In [ ]:
g_fwd.compute_schur(n_components=20)
g_fwd.plot_spectrum(real_only=True)

In [ ]:
g_fwd.compute_macrostates(n_states=10, cluster_key="celltypes")
g_fwd.plot_macrostates( which="all",
    discrete=True, legend_loc="right", size=100, basis="umap", title="potential terminal macrostates on embryonic tail",
    save="ET_macrostate_celltype_velokernel.png",
)

In [ ]:
g_fwd.plot_coarse_T(title="coarse-grained transition matrix of terminal macrostates", save="ET_macrostatecoarse_celltype_velokernel.png")

In [ ]:
g_fwd.plot_macrostate_composition(key="celltypes", figsize=(7, 4), save="ET_distribution_celltype_velokernel.png")

In [ ]:
#g_fwd.compute_macrostates(n_states=5, cluster_key="celltypes")
g_fwd.set_terminal_states(states=["Chondrocytes", "Myocytes", "Endothelial cells", "Neurons_1"])

In [ ]:
g_fwd.plot_macrostates(which="terminal", same_plot=True, basis="draw_graph_fr", 
                       legend_loc="right", title= "assigned terminal states", discrete=True ,
                           save="ET_ssignedterminalstate_fr_celltype_velokernel.png")

In [ ]:
g_fwd.plot_macrostates(which="terminal", same_plot=True, basis="draw_graph_lgl", 
                       legend_loc="right", title= "assigned terminal states", discrete=True ,
                           save="ET_ssignedterminalstate_lgl_celltype_velokernel.png")

In [ ]:
g_fwd.plot_macrostates(which="terminal", same_plot=True, basis="draw_graph_lgl", 
                       legend_loc="right", title= "assigned terminal states", discrete=False ,
                           save="ET_ssignedterminalstate_lgl_celltype_velokernel2.png")

In [ ]:
g_fwd.plot_macrostates(which="terminal", same_plot=True, basis="diffmap", 
                       legend_loc="right", title= "assigned terminal states", discrete=True ,
                           save="ET_ssignedterminalstate_diffmap_celltype_velokernel.png")

In [ ]:
g_fwd.plot_macrostates(which="terminal", same_plot=True, basis="diffmap", 
                       legend_loc="right", title= "assigned terminal states", discrete=False ,
                           save="ET_ssignedterminalstate_diffmap_celltype_velokernel2.png")

In [ ]:
g_fwd.plot_macrostates(which="terminal", same_plot=True, basis="umap", 
                       legend_loc="right", title= "assigned terminal states", discrete=True ,
                           save="ET_ssignedterminalstate_umap_celltype_velokernel.png")

In [ ]:
g_fwd.plot_macrostates(which="terminal", same_plot=True, basis="umap", 
                       legend_loc="right", title= "assigned terminal states", discrete=False ,
                           save="ET_ssignedterminalstate_umap_celltype_velokernel2.png")

In [ ]:
g_fwd.compute_fate_probabilities()
g_fwd.plot_fate_probabilities(same_plot=False, size=100, basis="draw_graph_lgl", ncols=1, 
                                    save="ET_absorbfate_lgl_celltype_velokernel.png",
)

In [ ]:
g_fwd.plot_fate_probabilities(same_plot=False, size=100, basis="diffmap", ncols=1, 
                                    save="ET_absorbfate_diffmap_celltype_velokernel.png",
)

In [ ]:
g_fwd.plot_fate_probabilities(same_plot=False, size=100, basis="umap", ncols=1, 
                                    save="ET_absorbfate_umap_celltype_velokernel.png",
)

In [ ]:
#g_fwd.predict_initial_states(allow_overlap=True)
#g_fwd.plot_macrostates(which="initial", legend_loc="right", s=100)

In [ ]:
Chondro_drivers = g_fwd.compute_lineage_drivers(lineages="Chondrocytes", return_drivers=True)
Chondro_drivers.sort_values(by="Chondrocytes_corr", ascending=False)

In [ ]:
g_fwd.plot_lineage_drivers("Chondrocytes", 
                           basis="draw_graph_fr", 
                           n_genes=12, ncols=4, 
                           save="ET_lineagedriver_Chondrocytes_velokernel.png")

In [ ]:
g_fwd.plot_lineage_drivers("Chondrocytes", 
                           basis="draw_graph_lgl", 
                           n_genes=12, ncols=4, 
                           save="ET_lineagedriver_Chondrocytes_lgl_velokernel.png")

In [ ]:
g_fwd.plot_lineage_drivers("Chondrocytes", 
                           basis="diffmap", 
                           n_genes=12, ncols=4, 
                           save="ET_lineagedriver_Chondrocytes_diffmap_velokernel.png")

In [ ]:
Neuron_drivers = g_fwd.compute_lineage_drivers(lineages="Neurons_1", return_drivers=True)
Neuron_drivers.sort_values(by="Neurons_1_corr", ascending=False)

In [ ]:
g_fwd.plot_lineage_drivers("Neurons_1", 
                           basis="draw_graph_lgl", 
                           n_genes=12, ncols=4, 
                           save="ET_lineagedriver_Neurons_lgl_velokernel.png")

In [ ]:
g_fwd.plot_lineage_drivers("Neurons_1", 
                           basis="diffmap", 
                           n_genes=12, ncols=4, 
                           save="ET_lineagedriver_Neurons_diffmap_velokernel.png")

In [ ]:
#cart_drivers = g_fwd.compute_lineage_drivers(lineages="Cartilage progenitor cell", return_drivers=True)
#cart_drivers.sort_values(by="Cartilage progenitor cell_corr", ascending=False)

In [ ]:
cr.pl.circular_projection(adata_merge, keys=["kl_divergence","celltypes"], figsize=[15,5] ,
                          legend_loc="right", ncols=2,
                          save="ET_circularprojection_velokernel.png")

In [ ]:
vk2 = VelocityKernel(adata_merge, backward=True).compute_transition_matrix()
print(vk)

In [ ]:
vk2.plot_projection(basis="draw_graph_fr", color=["celltypes","orig.ident"], 
                     title="backward velocity stream of regenerating tail", 
                     save="ET_embedstream_fr_celltype_bwd_cytokernel.png")

In [ ]:
vk2.plot_projection(basis="draw_graph_lgl", color=["celltypes","orig.ident"], 
                     title="backward velocity stream of regenerating tail", legend_loc="best",
                     save="ET_embedstream_lgl_celltype_bwd_cytokernel.png")

In [ ]:
vk2.plot_projection(basis="diffmap", color=["celltypes","orig.ident"], 
                     title="backward velocity stream of regenerating tail", legend_loc="best",
                     save="ET_embedstream_diffmap_celltype_bwd_cytokernel.png")

In [ ]:
vk2.plot_projection(basis="umap", color=["celltypes","orig.ident"], 
                     title="backward velocity stream of regenerating tail",
                     save="ET_embedstream_umap_celltype_bwd_cytokernel.png")

In [ ]:
g_bwd = GPCCA(vk2)
print(g_bwd)

In [ ]:
g_bwd.compute_schur(n_components=20)
g_bwd.plot_spectrum(real_only=True)

In [ ]:
g_bwd.compute_macrostates(n_states=9, cluster_key="celltypes")
g_bwd.plot_macrostates( which="all",
    discrete=True, legend_loc="right", size=100, basis="umap", title="potential initial macrostates on embryonic tail",
    save="ET_macrostate_init_stage_celltype_velokernel.png",
)

In [ ]:
g_bwd.plot_coarse_T(title="coarse-grained transition matrix of initial macrostates", save="ET_macrostatecoarse_celltype_bwd_velokernel.png")

In [ ]:
g_bwd.set_terminal_states(states=["Tailbud mesenchymal cells", "Neuroblasts"])

In [ ]:
g_bwd.plot_macrostates( which="terminal",
    discrete=True, legend_loc="right", size=100, basis="umap", title="Assigned initial macrostates on embryonic tail",
    save="ET_assignedinitstage_celltype_velokernel.png",
)

In [ ]:
g_bwd.plot_macrostates( which="terminal",
    discrete=True, legend_loc="right", size=100, basis="draw_graph_lgl", title="Assigned initial macrostates on embryonic tail",
    save="ET_assignedinitstage_lgl_celltype_velokernel.png",
)

In [ ]:
g_bwd.compute_fate_probabilities(tol=1e-8, solver='direct')
g_bwd.plot_fate_probabilities(same_plot=False, size=50, basis="draw_graph_lgl", ncols=2,
                                    save="ET_fateprob_initialstate_lgl_cytokernel.png")

In [ ]:
g_bwd.plot_fate_probabilities(same_plot=False, size=50, basis="diffmap", ncols=2,
                                    save="ET_fateprob_initialstate_diffmap_cytokernel.png")

In [ ]:
scv.tl.recover_dynamics(adata_merge, n_jobs=20)

In [ ]:
scv.tl.recover_latent_time(
    adata_merge,  root_key="term_states_bwd_probs", end_key="term_states_fwd_probs"
)

In [ ]:
scv.tl.paga(
    adata_merge,
    groups="celltypes",
    root_key="term_states_bwd_probs",
    end_key="term_states_fwd_probs",
    use_time_prior="velocity_pseudotime",
)

In [ ]:
cr.pl.aggregate_fate_probabilities(adata_merge, 
                                         mode="paga_pie", 
                                         backward=False, lineages=None, 
                                         cluster_key='celltypes', clusters=None, 
                                         basis="draw_graph_lgl", cbar=True, 
                                         ncols=None, sharey=False, fmt='0.2f', xrot=90,
                                         legend_kwargs={'loc':  "top right out"},
                                         title="directed PAGA",
                                         save="ET_directed_PAGA_lgl_velokernel.png")

In [ ]:
cr.pl.aggregate_fate_probabilities(adata_merge, 
                                         mode="paga_pie", 
                                         backward=False, lineages=None, 
                                         cluster_key='celltypes', clusters=None, 
                                         basis="draw_graph_fr", cbar=True, 
                                         ncols=None, sharey=False, fmt='0.2f', xrot=90, 
                                         legend_kwargs={'loc':  "top right out"},
                                         title="directed PAGA",
                                         save="ET_directed_PAGA_fr_velokernel.png")

In [ ]:
cr.pl.aggregate_fate_probabilities(adata_merge, 
                                         mode="paga_pie", 
                                         backward=False, lineages=None, 
                                         cluster_key='celltypes', clusters=None, 
                                         basis="diffmap", cbar=True, 
                                         ncols=None, sharey=False, fmt='0.2f', xrot=90, fontsize=0,
                                         legend_kwargs={'loc':  "top right out"},
                                         title="directed PAGA",
                                         save="ET_directed_PAGA_diffmap_velokernel.png")

In [ ]:
cr.pl.aggregate_fate_probabilities(adata_merge, 
                                         mode="paga_pie", 
                                         backward=False, lineages=None, 
                                         cluster_key='celltypes', clusters=None, 
                                         basis="draw_graph_drl", cbar=True, 
                                         ncols=None, sharey=False, fmt='0.2f', xrot=90, 
                                         legend_kwargs={'loc':  "top right out"},
                                         title="directed PAGA",
                                         save="ET_directed_PAGA_drl_velokernel.png")

In [ ]:
cr.pl.aggregate_fate_probabilities(
    adata_merge,
    mode="violin",
    lineages=["Chondrocytes", "Myocytes","Endothelial cells", "Neurons_1"],
    cluster_key="celltypes", ncols=4, #figsize=[5,15] # xrot=45, 
    save="ET_fate_probs_violin_velokernel.png"
)

In [ ]:
cr.pl.aggregate_fate_probabilities(
    adata_merge,
    mode="heatmap",
    lineages=["Chondrocytes", "Myocytes","Endothelial cells", "Neurons_1"],
    cluster_key="celltypes", ncols=3, #figsize=[5,15] # xrot=45, 
    save="ET_fate_probs_heatmap_velokernel.png",
)

In [ ]:
model = cr.models.GAM(adata_merge)

In [ ]:
cr.pl.heatmap(
    adata_merge,
    model=model,  # use the model from before
    lineages="Chondrocytes",
    cluster_key="celltypes",
    show_fate_probabilities=True,
    data_key="magic_imputed_data",
    genes=Chondro_drivers.head(40).index,
    time_key="ct_pseudotime",
    figsize=(12, 10),
    show_all_genes=True,
    weight_threshold=(1e-3, 1e-3),
)

In [ ]:
cr.pl.gene_trends(
    adata_merge,
    model,
    ["SHH","COL2A1","MYOG","MYOD1","CKM","MYL11","TNNT2","DES","SPP1","COL1A1","COL1A2"],
    data_key="Ms",
    same_plot=True,
    hide_cells=True,
    time_key="ct_pseudotime",
    show_progress_bar=False, save="ET_genetrend_2.png",
)

In [ ]:
adata_merge

In [ ]:
sc.pl.violin(adata_merge, keys=["latent_time"], groupby="customclassif", rotation=90, save="ET_cellrank_ctpseudotime_celltype.png")